In [92]:
import pandas as pd
import statsmodels.api as sm

tommy_johns = pd.read_csv('Tommy_John_Surgery_List.csv', skiprows = 1, usecols = ["Player"]).drop_duplicates()
pitch_velocity = pd.read_csv('savant_data_pitch_velocity.csv', usecols = ["player_name", "velocity", "total_pitches", "spin_rate"])
tommy_johns = tommy_johns.dropna(how = 'all')

tommy_johns['Player'] = tommy_johns['Player'].apply(lambda x: ', '.join(x.split()[::-1]))
tommy_johns['Had_TJ'] = 1

result = pd.merge(pitch_velocity, tommy_johns, how = 'left', left_on = ['player_name'], right_on = ['Player'])

result['TJ'] = result['Had_TJ'].notna().astype(int)
result = result[['player_name', 'velocity', 'TJ', 'total_pitches', 'spin_rate']]
result = result.dropna(subset = ['spin_rate'])
#result = result[result['total_pitches']>1000]

model = sm.Logit(result['TJ'], result[['velocity', 'total_pitches', 'spin_rate']]).fit()
model.summary()


Optimization terminated successfully.
         Current function value: 0.617988
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     TJ   No. Observations:                 2106
Model:                          Logit   Df Residuals:                     2103
Method:                           MLE   Df Model:                            2
Date:                Tue, 21 May 2024   Pseudo R-squ.:               -0.001209
Time:                        22:38:32   Log-Likelihood:                -1301.5
converged:                       True   LL-Null:                       -1299.9
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
velocity         -0.0131      0.008     -1.727      0.084      -0.028       0.002
total_pitches  1.289e-05   6.99e-06      1.844      0.065   -8.07e-07    2.66e-05
spin_rate         0.0002      0.000      0.504      0.614      -0.000       0.001
=================================================================================
"""

In [96]:
import pandas as pd
import matplotlib.pyplot as plt

tommy_johns = pd.read_csv('Tommy_John_Surgery_List.csv', skiprows = 1, usecols = ["Year"])
tommy_johns = tommy_johns.dropna(how = 'all')

#counts = tommy_johns['Year'].value_counts(sort = False).reset_index()
tommy_johns


#fig, ax = plt.subplots()
#ax.plot(counts['Year'], counts['count'], color="black")


,Year
0,2024.0
1,2024.0
2,2024.0
3,2024.0
4,2024.0
...,...
2439,1981.0
2440,1981.0
2441,1981.0
2442,1978.0
